In [1]:
from selenium import webdriver as wd
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from bs4 import BeautifulSoup
from urllib.request import urlopen

import pandas as pd
import numpy as np

import time

In [2]:
driver = wd.Chrome('chromedriver.exe')
saramin_filters = 'https://www.saramin.co.kr/zf_user/jobs/list/job-category?cat_kewd=82%2C257&searchType=search&searchword=%EB%8D%B0%EC%9D%B4%ED%84%B0+%EB%B6%84%EC%84%9D&exp_cd=1&edu_none=y&job_type=1&edu_min=6&edu_max=11&panel_type=&search_optional_item=y&search_done=y&panel_count=y&preview=y'
driver.get(saramin_filters)

C:\Users\kjaeh\AppData\Local\Temp\ipykernel_15876\1124558033.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = wd.Chrome('chromedriver.exe')


In [3]:
# DataFrame 생성
columns = ['기업명', '기업구분', '사원수', '설립일(업력)', '홈페이지', '지역']
corporations = pd.DataFrame(columns=columns)

In [7]:
# 각종 기능 함수
def click_btn_num(num): # 페이지 넘기기
    driver.find_element(By.CSS_SELECTOR, f'#default_list_wrap > div.pagination > a:nth-child({num})').click()

def get_soup(): # soup 정보 호출
    return BeautifulSoup(driver.page_source,'html.parser')

def get_info(soup):
    company_name = soup.find('span', 'company_name').get_text()

    # Empty 변수 생성 - 해당 값이 없는 경우를 대비
    corp_cat = ''; employee_num = 0; foundation_yr = 0; home_url = ''; corp_loc = ''

    # 'dl' 태그를 돌며 정보 수집
    for info in soup.find_all('dl'):
        if info.find('dt').get_text() == '기업형태':
            corp_cat = info.find('dd').get_text()
        
        elif info.find('dt').get_text() == '사원수':
            employee_num = info.find('dd').get_text()
        
        elif info.find('dt').get_text() == '설립일':
            foundation_yr = info.find('dd').get_text()

        elif info.find('dt').get_text() == '홈페이지':
            home_url = info.find('dd').get_text().lstrip('\n ').rstrip(' ')

        elif info.find('dt').get_text() == '기업주소':
            corp_loc = info.find('dd').get_text()
        
        else: 
            pass
    
    # 리스트에 정리 후 리스트 출력
    info_list = [company_name, corp_cat, employee_num, foundation_yr, home_url, corp_loc]
    return info_list

In [10]:
# 페이지 별 정보 수집
for i in range(2, 10):
    table = WebDriverWait(driver, 3).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, '#default_list_wrap > section > div.list_body')))
    contents = table.find_elements(By.CLASS_NAME, 'list_item')
    
    for c in contents:
        # 하이퍼링크 클릭
        c.find_element(By.TAG_NAME, 'a').click()
        driver.implicitly_wait(10)
        driver.switch_to.window(driver.window_handles[1])

        # 기업정보 버튼 클릭
        try:
            corp_btn = WebDriverWait(driver, 10).until(
                        EC.presence_of_element_located((By.XPATH, '//*[@id="content"]/div[4]/div[1]/button[4]'))) 
            corp_btn.click()
        except: # 기업 정보 버튼이 없을 시 탭을 종료한다
            driver.close()
            driver.switch_to.window(driver.window_handles[0])
            continue


        # 페이지 기준으로 soup 객체 생성
        time.sleep(3)
        soup = get_soup()
        corp_info = soup.find_all('div', 'wrap_info')[0] # 아래로 스크롤할 시 기업이 추가적으로 있으므로, 첫번째 기업만 추출
        
        # 앞서 지정한 함수에 따라 정보 DataFrame에 추가
        corporations.loc[len(corporations)+1] = get_info(corp_info)
        print(corporations.iloc[-1, :])

        # 탭 닫기
        driver.close()
        driver.switch_to.window(driver.window_handles[0])
    
    if i == 9:
        break
    else:
        click_btn_num(i)

기업명                        동아출판(주)
기업구분                  중견기업, 외부감사법인
사원수              283 명  (2022년 기준)
설립일(업력)     2008년 10월 1일 (업력 14년) 
홈페이지       www.dongapublishing.com
지역                  서울 영등포구 은행로 30
Name: 1, dtype: object
기업명                                              (주)페이타랩
기업구분                            중소기업, 스타트업, 주식회사, 병역특례..
사원수                                     52 명  (2022년 기준)
설립일(업력)                            2018년 2월 28일 (업력 4년) 
홈페이지                                     passorder.co.kr
지역         부산 부산진구 서면로 39, 6층 쿨리지코너부산센터 601호, 602호, 603호
Name: 2, dtype: object
기업명                      고려해운(주)
기업구분                중견기업, 외부감사법인
사원수            739 명  (2022년 기준)
설립일(업력)    1985년 9월 6일 (업력 37년) 
홈페이지              www.kmtc.co.kr
지역                 서울 중구 남대문로 63
Name: 3, dtype: object
기업명                       (주)엠제이플렉스
기업구분       중소기업, 스타트업, 외부감사법인, 주식..
사원수               930 명  (2022년 기준)
설립일(업력)      2005년 3월 24일 (업력 17년) 
홈페이지               www.mjplex.co.kr
지

In [21]:
driver.close()
driver.switch_to.window(driver.window_handles[0])

In [172]:
columns = ['기업명', '기업구분', '사원수', '설립일(업력)', '홈페이지', '지역']

corporations = pd.DataFrame(columns=columns)


,기업명,지역,기업구분,사원수,설립일(업력),홈페이지


In [11]:
corporations.head()

,기업명,기업구분,사원수,설립일(업력),홈페이지,지역
1,동아출판(주),"중견기업, 외부감사법인",283 명 (2022년 기준),2008년 10월 1일 (업력 14년),www.dongapublishing.com,서울 영등포구 은행로 30
2,(주)페이타랩,"중소기업, 스타트업, 주식회사, 병역특례..",52 명 (2022년 기준),2018년 2월 28일 (업력 4년),passorder.co.kr,"부산 부산진구 서면로 39, 6층 쿨리지코너부산센터 601호, 602호, 603호"
3,고려해운(주),"중견기업, 외부감사법인",739 명 (2022년 기준),1985년 9월 6일 (업력 37년),www.kmtc.co.kr,서울 중구 남대문로 63
4,(주)엠제이플렉스,"중소기업, 스타트업, 외부감사법인, 주식..",930 명 (2022년 기준),2005년 3월 24일 (업력 17년),www.mjplex.co.kr,"서울 구로구 디지털로33길 11, 406호"
5,에이에스이코리아(주),"외국인 투자기업, 외부감사법인, 수출입 ..",1885 명 (2021년 기준),1967년 3월 7일 (업력 55년),www.asekr.com,경기 파주시 산업단지길 76


In [ ]:
for i, r in corporations.iterrows():
    corporations.loc[i, '지역'] = ' '.join(r['지역'].split(' ')[:2])

In [31]:
corporations.to_csv('001_corporations.csv', sep=',', encoding='cp949')

#### 데이터 셋 Description

In [32]:
corporations = pd.read_csv('001_corporations.csv', encoding='cp949')
corporations.head()

,Unnamed: 0.1,Unnamed: 0,기업명,기업구분,사원수,설립일(업력),홈페이지,지역
0,0,1,동아출판(주),"중견기업, 외부감사법인",283 명 (2022년 기준),2008년 10월 1일 (업력 14년),www.dongapublishing.com,서울 영등포구
1,1,2,(주)페이타랩,"중소기업, 스타트업, 주식회사, 병역특례..",52 명 (2022년 기준),2018년 2월 28일 (업력 4년),passorder.co.kr,부산 부산진구
2,2,3,고려해운(주),"중견기업, 외부감사법인",739 명 (2022년 기준),1985년 9월 6일 (업력 37년),www.kmtc.co.kr,서울 중구
3,3,4,(주)엠제이플렉스,"중소기업, 스타트업, 외부감사법인, 주식..",930 명 (2022년 기준),2005년 3월 24일 (업력 17년),www.mjplex.co.kr,서울 구로구
4,4,5,에이에스이코리아(주),"외국인 투자기업, 외부감사법인, 수출입 ..",1885 명 (2021년 기준),1967년 3월 7일 (업력 55년),www.asekr.com,경기 파주시


In [33]:
corporations.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 368 entries, 0 to 367
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Unnamed: 0.1  368 non-null    int64 
 1   Unnamed: 0    368 non-null    int64 
 2   기업명           368 non-null    object
 3   기업구분          356 non-null    object
 4   사원수           368 non-null    object
 5   설립일(업력)       368 non-null    object
 6   홈페이지          317 non-null    object
 7   지역            368 non-null    object
dtypes: int64(2), object(6)
memory usage: 23.1+ KB


In [ ]:
corporations['지역'].unique()

array(['서울 영등포구', '부산 부산진구', '서울 중구', '서울 구로구', '경기 파주시', '-', '서울 송파구',
       '서울 강남구', '서울 마포구', '경기 성남시', '서울 은평구', '경기 안양시', '부산 중구',
       '서울 성동구', '부산 해운대구', '서울 금천구', '충북 청주시', '충남 아산시', '서울 종로구',
       '경기 용인시', '서울 서초구', '제주 제주시', '대전 서구', '서울 강서구', '대전 유성구',
       '부산 사상구', '경기 광명시', '서울 관악구', '경남 창원시', '서울 광진구', '대구 동구',
       '서울 양천구', '경기 안산시', '경기 군포시', '서울 서대문구', '경기 화성시', '경기 이천시',
       '서울 강동구', '경기 수원시', '경기 부천시', '광주 북구', '서울 동작구', '서울 용산구',
       '대구 수성구', '서울 동대문구', '전남 영광군', '충북 제천시', '부산 기장군', '경북 포항시',
       '대구 달서구', '경기 고양시', '경남 거제시', '인천 연수구', '인천 서구', '충남 천안시',
       '경기 의정부시', '대구 남구', '대전 동구', '경남 양산시', '경기 하남시', '부산 강서구', '대구 중구',
       '인천 중구', '경남 밀양시', '인천 부평구', '전북 익산시'], dtype=object)

In [20]:
len(corporations['기업명'].unique())

263

In [18]:
corporations['사원수'].value_counts()

0                    174
133 명  (2022년 기준)      6
32 명  (2022년 기준)       5
900 명  (2022년 기준)      4
27 명  (2022년 기준)       4
                    ... 
635 명  (2021년 기준)      1
524 명  (2022년 기준)      1
106 명  (2022년 기준)      1
26 명  (2018년 기준)       1
12 명  (2018년 기준)       1
Name: 사원수, Length: 128, dtype: int64

In [16]:
corporations['설립일(업력)'].value_counts()

0                         147
2006년 10월 2일 (업력 16년)       6
2018년 6월 19일 (업력 4년)        5
2022년 2월 4일 (업력 0년)         4
2016년 1월 28일 (업력 6년)        4
                         ... 
2007년 1월 2일 (업력 15년)        1
2001년 3월 17일 (업력 21년)       1
2021년 9월 9일 (업력 1년)         1
2013년 8월 27일 (업력 9년)        1
2017년 3월 23일 (업력 5년)        1
Name: 설립일(업력), Length: 159, dtype: int64